<a href="https://colab.research.google.com/github/ftk1000/BERT_demos/blob/master/BertPunc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Edit -> Notebook setting -> hardware accelerator -> GPU

# Timing

In [0]:
# for the sake of convenience you can create folder in google drive disk, and put all files from archives to: C:\Users\USER_NAME\Google Drive\Colab Notebooks\Punctuation_Restoration\For_Farid

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ls -l

total 8
drwx------ 4 root root 4096 Feb 12 23:02 drive
drwxr-xr-x 1 root root 4096 Feb  5 18:37 sample_data


In [0]:
!ls -l drive/My\ Drive/For_Farid
%cd drive/My\ Drive/For_Farid

total 141487
-rw------- 1 root root     41361 Feb 11 20:48 7k_without_punctuation.txt
-rw------- 1 root root    228754 Feb 12 23:02 BertPunc.ipynb
-rw------- 1 root root      1034 Feb 11 20:48 convert_to_readable.py
-rw------- 1 root root     10427 Feb 11 20:48 data.py
-rw------- 1 root root      8324 Feb 11 20:48 data.pyc
-rw------- 1 root root 144456098 Feb 11 20:51 Demo-Europarl-EN.pcl
-rw------- 1 root root      6735 Feb 11 20:48 main.py
-rw------- 1 root root       656 Feb 11 20:48 model.py
-rw------- 1 root root     12155 Feb 11 20:48 models.py
-rw------- 1 root root     13876 Feb 11 20:48 models.pyc
-rw------- 1 root root     68553 Feb 11 20:48 Punctuator2.ipynb
-rw------- 1 root root      5944 Feb 11 20:48 punctuator.py
drwx------ 2 root root      4096 Feb 11 20:48 __pycache__
drwx------ 2 root root      4096 Feb 11 20:48 results
-rw------- 1 root root     16827 Feb 11 20:48 timing.docx
/content/drive/My Drive/For_Farid


In [0]:
###%cd drive/My\ Drive/Colab\ Notebooks/Punctuation_Restoration/For_Farid

In [0]:
#!dir

In [0]:
!pip install pytorch_pretrained_bert

     |████████████████████████████████| 133kB 8.8MB/s 


In [0]:
import pandas as pd
import numpy as np
from glob import glob
from pytorch_pretrained_bert import BertTokenizer
import torch
from torch import nn
%matplotlib inline
import json
from tqdm import tqdm
from sklearn import metrics
from model import BertPunc
# from data import load_file, preprocess_data, create_data_loader
from torch.utils.data import TensorDataset, DataLoader

In [0]:
print(torch.__version__)

1.4.0


In [0]:
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P4


In [0]:
batch_size = 1024
batch_size=128

In [0]:
glob('results/*')

['results/20200206_041031', 'results/20200205_012725']

In [0]:
path = 'results/20200205_012725/'

In [0]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
with open(path+'hyperparameters.json', 'r') as f:
    hyperparameters = json.load(f)
hyperparameters
punctuation_enc = {
    'O': 0,
    'COMMA': 1,
    'PERIOD': 2,
    'QUESTION': 3
}

segment_size = hyperparameters['segment_size']

In [0]:
def load_file(filename):
    with open(filename, 'r', encoding="utf8", errors="ignore") as f:  # , encoding='utf-8'
        data = f.readlines()
    return data

In [0]:
data_test = load_file('7k_without_punctuation.txt')
print(type(data_test))
print(len(data_test))
print(data_test[0][:50])

<class 'list'>
1
To get here we'd spent a day riding motorbikes ove


In [0]:
X = []
for line in data_test:
    # word = line.split()
    # print(word)
    tokens = tokenizer.tokenize(line)
    print(len(tokens))
    print(line[:10])
    x = tokenizer.convert_tokens_to_ids(tokens)
    if len(x) > 0:
        X += x
print(X[:10])
print(len(X))

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (8112 > 512). Running this sequence through BERT will result in indexing errors


8112
To get her
[2000, 2131, 2182, 2057, 1005, 1040, 2985, 1037, 2154, 5559]
8112


In [0]:
print(' Original: ', data_test[0][:50])
print('Tokenized: ', tokens[:10])
print('Token IDs: ', X[:10])

 Original:  To get here we'd spent a day riding motorbikes ove
Tokenized:  ['to', 'get', 'here', 'we', "'", 'd', 'spent', 'a', 'day', 'riding']
Token IDs:  [2000, 2131, 2182, 2057, 1005, 1040, 2985, 1037, 2154, 5559]


In [0]:
def insert_target(x, segment_size):

    X = []
    x_pad = x[-((segment_size-1)//2-1):]+x+x[:segment_size//2]

    for i in range(len(x_pad)-segment_size+2):
        segment = x_pad[i:i+segment_size-1]
        segment.insert((segment_size-1)//2, 0)
        X.append(segment)

    return np.array(X)

In [0]:
X_zero_halfway = insert_target(x, segment_size)

In [0]:
print(type(X_zero_halfway))
X_zero_halfway.shape

<class 'numpy.ndarray'>


(8112, 32)

In [0]:
# convert to torch's internal format
data_set = TensorDataset(torch.from_numpy(X_zero_halfway).long())

In [0]:
print(type(data_set))

<class 'torch.utils.data.dataset.TensorDataset'>


what is a batch? we think it is a matrix of size 1024x32, each entry is a code-word

In [0]:
print(batch_size)

128


In [0]:
data_loader = DataLoader(data_set, batch_size, shuffle=False)

In [0]:
type(data_loader)

torch.utils.data.dataloader.DataLoader

In [0]:
print(punctuation_enc)

{'O': 0, 'COMMA': 1, 'PERIOD': 2, 'QUESTION': 3}


In [0]:
output_size = len(punctuation_enc)

In [0]:
dropout = hyperparameters['dropout']

In [0]:
# from torch import nn
# from model import BertPunc
bert_punc = nn.DataParallel(BertPunc(segment_size, output_size, dropout).cuda())

In [0]:
BertPunc

model.BertPunc

In [0]:
type(bert_punc)

torch.nn.parallel.data_parallel.DataParallel

In [0]:
# what is this doing?????

# bert_punc.load_state_dict(torch.load(path+'model'))

In [0]:
# evaluate parallelized Bert_Punc
bert_punc.eval();

In [0]:
for batch in data_loader:
    # print(batch)
    print(type(batch))
    # print(batch[0])
    print(type(batch[0]))
    break

<class 'list'>
<class 'torch.Tensor'>


In [0]:
import time
start_time = time.time()

pred = []
for inputs in tqdm(data_loader):
    with torch.no_grad():
        inputs = inputs[0].cuda()
        output = bert_punc(inputs)
        pred += list(output.argmax(dim=1).cpu().data.numpy().flatten())

elapsed_time = time.time() - start_time
print(time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))

100%|██████████| 64/64 [00:23<00:00,  3.27it/s]

00:00:23


In [0]:
print(pred[:20])

[2, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3]


In [0]:
pred.count(0)

12

In [0]:
pred.count(1)

2676

In [0]:
pred.count(2)

558

In [0]:
# [to be continued]